In [11]:
!pip install groq

In [ ]:
# 🧠 HSN Code Validation Agent using LangChain + Groq + LLaMA 3.3
import os
import pandas as pd
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain_groq import ChatGroq



In [13]:
# ✅ Set your Groq API Key
os.environ["GROQ_API_KEY"] = "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"  # Replace with your key


In [14]:
# ✅ Load and Normalize the Excel File
df = pd.read_excel("HSN_SAC.xlsx")

# Show raw columns
print("📊 Raw columns:", df.columns.tolist())

# Normalize and clean column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "")

# Validate required columns
if "hsncode" not in df.columns or "description" not in df.columns:
    raise ValueError("❌ Excel file must contain columns: 'HSN Code' and 'Description'")

# Rename and index
df = df.rename(columns={"hsncode": "HSNCode", "description": "Description"})
df["HSNCode"] = df["HSNCode"].astype(str).str.strip()
df = df.set_index("HSNCode")


📊 Raw columns: ['\nHSNCode', 'Description']


In [15]:
# ✅ Define the Validation Function
def validate_hsn(code: str) -> str:
    code = code.strip()
    if not code.isdigit() or not (2 <= len(code) <= 8):
        return f"❌ '{code}' is an invalid format (must be numeric and 2–8 digits)."
    elif code in df.index:
        return f"✅ '{code}' is valid: {df.loc[code]['Description']}"
    else:
        # Check parent hierarchy
        for i in range(len(code) - 2, 1, -2):
            parent = code[:i]
            if parent in df.index:
                return f"⚠️ '{code}' not found. Closest match: '{parent}' → {df.loc[parent]['Description']}"
        return f"⚠️ '{code}' not found in master data."


In [16]:
# ✅ Create the LangChain Tool
hsn_tool = Tool(
    name="ValidateHSNCode",
    func=validate_hsn,
    description="Use this to validate an HSN code from the Excel master data. Input must be a single numeric code."
)


In [18]:
!pip install groq

# ✅ Set up the Groq LLM Agent (LLaMA 3.3 70B)
llm = ChatGroq(
    model="llama-3.3-70b-verstatile",
    temperature=0.2
)

agent = initialize_agent(
    tools=[hsn_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


ImportError: Could not import groq python package. Please install it with `pip install groq`.

In [19]:
from groq import Groq

client = Groq(api_key="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")

response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[{"role": "user", "content": "Hello!"}]
)

print(response.choices[0].message.content)


ImportError: cannot import name 'version_short' from 'pydantic.version' (c:\Users\LENOVO\anaconda3\lib\site-packages\pydantic\version.cp310-win_amd64.pyd)